In [1]:
## Importing all relevant libraries

import random

import numpy as np
import pandas as pd

from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn import svm
from sklearn import metrics
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score


from transformers import pipeline
from transformers import AutoTokenizer, AutoModel, AutoModelForPreTraining
from transformers import BigBirdConfig

import torch

import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

In [2]:
df = pd.read_excel(r"software_600.xlsx", index_col=0) ## Reading in data
#df_save = df.reset_index(inplace=False, drop = True)
df = df.reset_index(inplace=False, drop = True)
df

,an,title,snippet,body,full_text,"Does it mention an innovation (yes,no,unsure)?",What is the company?,"Where is company mentioned (title,body,snippet,all)?",What is the product name?,"Where is the product name mentioned(title,body,snippet,all)","What could indicate that this is a product (for use in our algorithms, e.g. TM,R,C,CAPITAL LETTERS, Quotes,Used with certain key phrases or words etc.)?","Features of the product, if mentioned",assignee,YN_INNOVATION
0,PNN0000020180426ee4p00006,Creaform and Metrologic Group Announce Worldwi...,Metrologic Group and Creaform have entered int...,Speed and volumetric accuracy\n\nThe CUBE-R™ e...,Creaform and Metrologic Group Announce Worldwi...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Diggy,yes
1,INVDAI0020150827eb8r00011,"4 Growth Stocks Make Case For ""New Leader"" Status",The IPO Leaders screen can hold up to 16 stock...,"Still, it makes sense to fill your watch list ...","4 Growth Stocks Make Case For ""New Leader"" Sta...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,Diggy,no
2,CNEWSN0020150403eb4200008,Microsoft puts kibosh on Kinect for Windows se...,Last year when Microsoft announced an adapter ...,"Starting today, Microsoft will no longer be ma...",Microsoft puts kibosh on Kinect for Windows se...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Diggy,no
3,NDRSNH0020200312eg3a00003,"County Council approves $850,000 for additiona...",ANDERSON — The Madison County Council approved...,"The additional $16,637 will cover the expenses...","County Council approves $850,000 for additiona...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,Diggy,no
4,LBA0000020200124eg1o02o7x,"Novartis, GBT sickle cell drugs too expensive,...",* U.S. group suggests SCD drugs too costly\n\n...,* Report may change after public comments\n\nB...,"Novartis, GBT sickle cell drugs too expensive,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,Diggy,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,WATERTWN20141208eac30001o,A REVIEW OF FOUR APPS THAT WILL HELP YOU SHOP ...,Want to save some cash while holiday shopping?...,I found four apps to recommend after testing m...,A REVIEW OF FOUR APPS THAT WILL HELP YOU SHOP ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aditi,no
596,TELWK00020130531e96500010,Transaction Network Services; TNS Unveils Uniq...,2013 JUN 5 (VerticalNews) -- By a News Reporte...,TNS has partnered with leading mobile and land...,Transaction Network Services; TNS Unveils Uniq...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aditi,yes
597,INTA000020140410ea4a00009,Red Hat introduces new OpenStack training course,"Red Hat, Inc. (NYSE: RHT) said it has announce...",The course is available via traditional and vi...,Red Hat introduces new OpenStack training cour...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aditi,no
598,ATRTAL0020200423eg4n000s8,"Dragontail Systems Enters U.S. Market, Launche...","NEW YORK, April 23 -- Dragontail Systems Limit...","Dragontail Systems Limited (ASX: DTS), a softw...","Dragontail Systems Enters U.S. Market, Launche...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aditi,yes


In [3]:
df_new = pd.read_csv("/home/asr8r/git/dspg21binn/data/DNA_Food_Articles_2017_no_rep.csv", index_col=0) ## Reading in data
#df_save = df.reset_index(inplace=False, drop = True)
df_new = df_new.reset_index(inplace=False, drop = True)
df_new = df_new.iloc[0:5000,] #Subsetting the data to only the first 5000 articles
df_new

,index,copyright,subject_codes,art,modification_datetime,body,company_codes_occur_ticker_exchange,company_codes_occur,company_codes_about,company_codes_lineage,...,company_codes,industry_codes,title,publication_datetime,publisher_name,action,document_type,date,year,full_text
0,15,© Copyright 2017 Journal of Engineering via Ve...,",ccat,",NaN,1489442488000,Those respondents with advanced full life cycl...,NaN,",comasc,",",comasc,",NaN,...,",comasc,comasc,comasc,",",imobsoft,i3302,i330202,i3302021,icomp,itech,",CA Technologies; Are You Getting the Most Valu...,1489363200000,"NewsRX, LLC",add,article,2017-03-13 00:00:00.000,2017,CA Technologies; Are You Getting the Most Valu...
1,18,© Copyright 2017 Computer Weekly News via via ...,",ccat,",NaN,1487890959000,The Access Valve for ThingWorx integrates an o...,NaN,NaN,NaN,NaN,...,NaN,",isecpri,iioft,i3302,i330202,i3302021,i3303,ic...","Certified Security Solutions, Inc. CSS Release...",1487808000000,"NewsRX, LLC",add,article,2017-02-23 00:00:00.000,2017,"Certified Security Solutions, Inc. CSS Release..."
2,26,Copyright 2017 Telecompaper. All Rights Reserv...,",ccat,c22,cprdop,c24,ncat,nfact,nfcpin,",NaN,1508513602000,Cortana on the new speaker will look up inform...,NaN,",mcrost,harmki,gognew,bstbuy,amzcom,",",mcrost,harint,bstbuy,harmki,",",sansel,",...,",harmki,amzcom,amzcom,bstbuy,bstbuy,gognew,gog...",",i34532,i3302,icnp,idurhg,ielec,ihome,itech,i3...",Harman Kardon launches smart speaker with Micr...,1508457600000,Telecompaper BV,add,article,2017-10-20 00:00:00.000,2017,Harman Kardon launches smart speaker with Micr...
3,36,"© Copyright 2017 GlobeNewswire, Inc. All Right...",",cpartn,ccpint,npress,c11,ccat,ncat,",NaN,1484229727000,"According to Lee Nagel, vice president of mark...",NaN,",orncmi,onlnfr,linkd,",NaN,NaN,...,",linkd,linkd,onlnfr,onlnfr,orncmi,orncmi,twnit,",",ibisw,i3302021,i3302,i330202,icomp,itech,",Izenda Partners with Orion Communications to P...,1484229604057,"GlobeNewswire, Inc.",add,article,2017-01-12 14:00:04.057,2017,Izenda Partners with Orion Communications to P...
4,57,© Copyright 2017 Computer Weekly News via via ...,",ccat,",NaN,1503007605000,Given its aggressive plans to take more market...,NaN,",hubspo,",",hubspo,",NaN,...,",hubspo,hubspo,hubspo,salesf,",",icomp,i3302,itech,i330202,i3302021,ientrps,",Qstream; HubSpot Signs Global Agreement with Q...,1502928000000,"NewsRX, LLC",add,article,2017-08-17 00:00:00.000,2017,Qstream; HubSpot Signs Global Agreement with Q...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,64678,© Copyright 2017 Telecommunications Weekly via...,",c133,cgymtr,ccat,cinprp,",NaN,1492034538000,The following quote was obtained by the news e...,NaN,NaN,NaN,NaN,...,NaN,",iappsp,i8394,ibcs,","Patents; ""System and Method for Capturing and ...",1491955200000,"NewsRX, LLC",add,article,2017-04-12 00:00:00.000,2017,"Patents; ""System and Method for Capturing and ..."
4996,64694,"Copyright 2017, Conley Publishing Group Ltd. A...",",gabus,cmerg,c18,c181,cacqu,cactio,ccat,gcat,g...",John Ehlke/Daily News The exterior of Elevate ...,1497953185000,Elevate Executive Director Mary Simon said the...,NaN,",hglbsv,",",hglbsv,",NaN,...,",hglbsv,hglbsv,hglbsv,",",i8394,ibcs,",Elevate kicks off fundraising campaign,1497916800000,"Conley Publishing Group, Ltd.",add,article,2017-06-20 00:00:00.000,2017,Elevate kicks off fundraising campaign jvan@co...
4997,64699,© Copyright 2017 Telecommunications Weekly via...,",c22,ccat,ncat,nfact,nfcpin,",NaN,1505340179000,FWD is a cloud-based solution that enables con...,NaN,",nokia,cmptl,aqnoci,",",cmptl,",",nokia,",...,",cmptl,aqnoci,aqnoci,cmptl,cmptl,linkd,nokia,n...",",i7902202,i7902,i79022,i3302,i330202,i3302020,...",Comptel Corporation; Viva Bolivia Launches Ili...,1505260800000,"NewsRX, LLC",add,article,2017-09-13 00:00:00.000,2017,Comptel Corporation; Viva Bolivia Launches Ili...
4998,64715,© Copyright 2017 Marketing Weekly News via Ver...,",cpartn,c31,ccat,ncat,

In [4]:
## Balancing target class 

df_yes = df[df.YN_INNOVATION == 'yes']
df_no = df[df.YN_INNOVATION == 'no']

if len(df_yes) == len(df_no):
    df_yes = df_yes.append(df_no)
    df = df_yes
    
elif len(df_yes) > len(df_no):
    df_yes = df_yes.sample(n=len(df_no), random_state = 65432)
    df_yes = df_yes.append(df_no)
    df = df_yes
    
elif len(df_no) > len(df_yes):
    df_no = df_no.sample(n=len(df_yes), random_state = 65432)
    df_yes = df_yes.append(df_no)
    df = df_yes
    
df = df.sample(n=len(df), random_state = 65432).reset_index(inplace=False, drop = True)

    
print("Value counts for target variable:\n",df.YN_INNOVATION.value_counts())

## Encoding target variable

le = preprocessing.LabelEncoder()
df.YN_INNOVATION = le.fit_transform(df.YN_INNOVATION)
print("\n\n\nThe target variable classes are:", le.classes_)

Value counts for target variable:
 yes    222
no     222
Name: YN_INNOVATION, dtype: int64



The target variable classes are: ['no' 'yes']


In [5]:
## NLP Models from Hugging Face, pretrained_weights object is changed according to required model
model_class, tokenizer_class, pretrained_weights = (AutoModel, 
                                                    AutoTokenizer, 
                                                    'distilbert-base-uncased')
                                                 
# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

#Tokenize
tokenized = df['full_text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True, padding=True, 
                                                          truncation=True,max_length=512
                                                             )))


#Train Data
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

np.array(padded).shape
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)
    
    
    


#Test/Train  
features = last_hidden_states[0][:,0,:].numpy()
labels = df['YN_INNOVATION']
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

#Logistic Regression
lr_clf = LogisticRegression(solver="sag")
lr_clf.fit(train_features, train_labels)

#Accuracy
lr_clf.score(test_features, test_labels)



Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/home/asr8r/.conda/envs/akbert/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


0.8468468468468469

In [6]:
tokenized_new =  df_new['full_text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True, padding=True, 
                                                          truncation=True,max_length=512)))

#New Data

max_len = 0
for i in tokenized_new.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized_new.values])

np.array(padded).shape
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states_new = model(input_ids, attention_mask=attention_mask)

features_new = last_hidden_states_new[0][:,0,:].numpy()




predicted_y = lr_clf.predict(features_new)
pred_prob = lr_clf.predict_proba(features_new)[:, 1]

df_new["predicted_Innovation"] = predicted_y
df_new


,index,copyright,subject_codes,art,modification_datetime,body,company_codes_occur_ticker_exchange,company_codes_occur,company_codes_about,company_codes_lineage,...,industry_codes,title,publication_datetime,publisher_name,action,document_type,date,year,full_text,predicted_Innovation
0,15,© Copyright 2017 Journal of Engineering via Ve...,",ccat,",NaN,1489442488000,Those respondents with advanced full life cycl...,NaN,",comasc,",",comasc,",NaN,...,",imobsoft,i3302,i330202,i3302021,icomp,itech,",CA Technologies; Are You Getting the Most Valu...,1489363200000,"NewsRX, LLC",add,article,2017-03-13 00:00:00.000,2017,CA Technologies; Are You Getting the Most Valu...,0
1,18,© Copyright 2017 Computer Weekly News via via ...,",ccat,",NaN,1487890959000,The Access Valve for ThingWorx integrates an o...,NaN,NaN,NaN,NaN,...,",isecpri,iioft,i3302,i330202,i3302021,i3303,ic...","Certified Security Solutions, Inc. CSS Release...",1487808000000,"NewsRX, LLC",add,article,2017-02-23 00:00:00.000,2017,"Certified Security Solutions, Inc. CSS Release...",1
2,26,Copyright 2017 Telecompaper. All Rights Reserv...,",ccat,c22,cprdop,c24,ncat,nfact,nfcpin,",NaN,1508513602000,Cortana on the new speaker will look up inform...,NaN,",mcrost,harmki,gognew,bstbuy,amzcom,",",mcrost,harint,bstbuy,harmki,",",sansel,",...,",i34532,i3302,icnp,idurhg,ielec,ihome,itech,i3...",Harman Kardon launches smart speaker with Micr...,1508457600000,Telecompaper BV,add,article,2017-10-20 00:00:00.000,2017,Harman Kardon launches smart speaker with Micr...,1
3,36,"© Copyright 2017 GlobeNewswire, Inc. All Right...",",cpartn,ccpint,npress,c11,ccat,ncat,",NaN,1484229727000,"According to Lee Nagel, vice president of mark...",NaN,",orncmi,onlnfr,linkd,",NaN,NaN,...,",ibisw,i3302021,i3302,i330202,icomp,itech,",Izenda Partners with Orion Communications to P...,1484229604057,"GlobeNewswire, Inc.",add,article,2017-01-12 14:00:04.057,2017,Izenda Partners with Orion Communications to P...,1
4,57,© Copyright 2017 Computer Weekly News via via ...,",ccat,",NaN,1503007605000,Given its aggressive plans to take more market...,NaN,",hubspo,",",hubspo,",NaN,...,",icomp,i3302,itech,i330202,i3302021,ientrps,",Qstream; HubSpot Signs Global Agreement with Q...,1502928000000,"NewsRX, LLC",add,article,2017-08-17 00:00:00.000,2017,Qstream; HubSpot Signs Global Agreement with Q...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,64678,© Copyright 2017 Telecommunications Weekly via...,",c133,cgymtr,ccat,cinprp,",NaN,1492034538000,The following quote was obtained by the news e...,NaN,NaN,NaN,NaN,...,",iappsp,i8394,ibcs,","Patents; ""System and Method for Capturing and ...",1491955200000,"NewsRX, LLC",add,article,2017-04-12 00:00:00.000,2017,"Patents; ""System and Method for Capturing and ...",0
4996,64694,"Copyright 2017, Conley Publishing Group Ltd. A...",",gabus,cmerg,c18,c181,cacqu,cactio,ccat,gcat,g...",John Ehlke/Daily News The exterior of Elevate ...,1497953185000,Elevate Executive Director Mary Simon said the...,NaN,",hglbsv,",",hglbsv,",NaN,...,",i8394,ibcs,",Elevate kicks off fundraising campaign,1497916800000,"Conley Publishing Group, Ltd.",add,article,2017-06-20 00:00:00.000,2017,Elevate kicks off fundraising campaign jvan@co...,0
4997,64699,© Copyright 2017 Telecommunications Weekly via...,",c22,ccat,ncat,nfact,nfcpin,",NaN,1505340179000,FWD is a cloud-based solution that enables con...,NaN,",nokia,cmptl,aqnoci,",",cmptl,",",nokia,",...,",i7902202,i7902,i79022,i3302,i330202,i3302020,...",Comptel Corporation; Viva Bolivia Launches Ili...,1505260800000,"NewsRX, LLC",add,article,2017-09-13 00:00:00.000,2017,Comptel Corporation; Viva Bolivia Launches Ili...,1
4998,64715,© Copyright 2017 Marketing Weekly News via Ver...,",cpartn,c31,ccat,ncat,nfact,nfcpin,",NaN,1487373426000,"""It's important for marketers to quickly and e...",NaN,",rkwrhd,hubspo,",",rkwrhd,hubspo,",NaN,...,",i8385,iadv,ibcs,imark,i3302,i330202,i3302021,...","Shutterstock, Inc. Shutterstock an